In [5]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

In [6]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

class PlainRAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)

        super(PlainRAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(PlainRAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                beta2_t = beta2 ** state['step']
                N_sma_max = 2 / (1 - beta2) - 1
                N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:                    
                    step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    step_size = group['lr'] / (1 - beta1 ** state['step'])
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss


class AdamW(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, warmup = 0):
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, warmup = warmup)
        super(AdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdamW, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                
                if group['warmup'] > state['step']:
                    scheduled_lr = 1e-8 + state['step'] * group['lr'] / group['warmup']
                else:
                    scheduled_lr = group['lr']

                step_size = scheduled_lr * math.sqrt(bias_correction2) / bias_correction1
                
                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * scheduled_lr, p_data_fp32)

                p_data_fp32.addcdiv_(-step_size, exp_avg, denom)

                p.data.copy_(p_data_fp32)

        return loss

In [7]:
!ls /mnt/chicm/data/bengali

bengaliai-cv19.zip	   test_image_data_3.parquet
class_map.csv		   train.csv
sample_submission.csv	   train_image_data_0.parquet
test.csv		   train_image_data_1.parquet
test_image_data_0.parquet  train_image_data_2.parquet
test_image_data_1.parquet  train_image_data_3.parquet
test_image_data_2.parquet


In [4]:
#!ls /home/chec/data/bengali

In [8]:
DATA_DIR = '/mnt/chicm/data/bengali'

In [9]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [7]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [10]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [9]:
#plt.imshow(x)

In [11]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

HEIGHT = 137
WIDTH = 236

class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            img = augs(image=img)['image']
        
        img = np.expand_dims(img, axis=-1)
        
        img = transforms.functional.to_tensor(img)
        #img = transforms.functional.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [12]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

# model

In [13]:
#import pretrainedmodels

In [14]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [15]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [16]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [17]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

In [18]:
#model.last_linear.in_features

In [19]:
class BengaliNet_1(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
    def logits(self, x):
        x = self.avg_pool(x)
        x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [20]:
class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        #self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        self.fix_input_layer()
        
    def fix_input_layer(self):
        if self.backbone_name in ['se_resnext50_32x4d', 'se_resnext101_32x4d', 'se_resnet50', 'senet154', 'se_resnet152', 'nasnetmobile', 'mobilenet', 'nasnetalarge']:
            #self.backbone = eval(backbone_name)()
            #print(self.backbone.layer0.conv1)
            w = self.backbone.layer0.conv1.weight.data
            self.backbone.layer0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            #self.backbone.layer0.conv1.weight = torch.nn.Parameter(torch.cat((w, w[:, 2, :, :].unsqueeze(1)), dim=1))
            self.backbone.layer0.conv1.weight = torch.nn.Parameter(w[:, 0, :, :].unsqueeze(1))
        
    def logits(self, x):
        x = self.avg_pool(x)
        x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        #x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [21]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [21]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [22]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [23]:
round(1/9, 6)

0.111111

In [22]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [23]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [24]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [25]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [26]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [27]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [28]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [29]:
np.random.random()

0.3693146998813017

In [30]:
from over9000.over9000 import Over9000

In [31]:
def train(args):
    global model

    if args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.0001)
    elif args.optim == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    elif args.optim == 'Over9000':
        optimizer = Over9000(model.parameters(), lr=args.lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.0001)

    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.t_max, eta_min=args.min_lr)
        
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    #if args.lrs == 'plateau':
    #    lr_scheduler.step(best_metrics)
    #else:
    #    lr_scheduler.step()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #do_mixup = False #(np.random.random() < 0.4)
            
            #if do_mixup:
            #    img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            
            
            #if do_mixup:
            #    loss = mixup_criterion(outputs, targets)
            #else:
            #    loss = criterion(outputs, targets)
            r = np.random.rand()
            #if args.beta > 0 and r < args.cutmix_prob:
            if r < 0.5:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs = model(img)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
                #img, targets = mixup(img, targets)
                outputs = model(img)
                #loss = mixup_criterion(outputs, targets)
                loss = criterion(outputs, targets)
            
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            
            #loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [37]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'model2_best_model.pth'
args.predict = False
args.optim = 'Over9000'
args.lr = 1e-4
args.lrs = 'cosine'
args.t_max = 15
args.factor = 0.6
args.patience = 2
args.min_lr = 2e-6
args.iter_val = 200
args.num_epochs = 100000
args.batch_size = 1024
args.val_batch_size = 1024

args.beta = 1.0
args.cutmix_prob = 0.5

In [33]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=2)

(200840, 5)
(200840, 32332)
(160678, 5) (40162, 5)


In [38]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


model file: ./models/se_resnext50_32x4d/model2_best_model.pth, exist: True
loading ./models/se_resnext50_32x4d/model2_best_model.pth...


In [ ]:
train(args)

{'recall': 0.983832, 'recall_grapheme': 0.976587, 'recall_vowel': 0.992964, 'recall_consonant': 0.989192, 'acc_grapheme': 0.975499, 'acc_vowel': 0.993725, 'acc_consonant': 0.992082, 'loss_grapheme': 0.10933, 'loss_vowel': 0.045842, 'loss_consonant': 0.039667}
    1 | 0.000100 | 045056/160678 | 3.8738 | 1.4469 |
val: {'recall': 0.983691, 'recall_grapheme': 0.976539, 'recall_vowel': 0.993165, 'recall_consonant': 0.988521, 'acc_grapheme': 0.975649, 'acc_vowel': 0.9938, 'acc_consonant': 0.992182, 'loss_grapheme': 0.107471, 'loss_vowel': 0.042411, 'loss_consonant': 0.038076}
    2 | 0.000099 | 090112/160678 | 0.2990 | 1.2315 |
val: {'recall': 0.983594, 'recall_grapheme': 0.976415, 'recall_vowel': 0.993047, 'recall_consonant': 0.9885, 'acc_grapheme': 0.976022, 'acc_vowel': 0.993775, 'acc_consonant': 0.992381, 'loss_grapheme': 0.105525, 'loss_vowel': 0.033101, 'loss_consonant': 0.033147}
    3 | 0.000096 | 135168/160678 | 0.0027 | 1.4807 |
val: {'recall': 0.983785, 'recall_grapheme': 0.975925

In [35]:
train(args) 

{'recall': 0.983759, 'recall_grapheme': 0.976406, 'recall_vowel': 0.992934, 'recall_consonant': 0.98929, 'acc_grapheme': 0.975425, 'acc_vowel': 0.993551, 'acc_consonant': 0.992157, 'loss_grapheme': 0.106273, 'loss_vowel': 0.041701, 'loss_consonant': 0.038099}
    1 | 0.000020 | 045056/160678 | 1.2768 | 1.7016 |
val: {'recall': 0.983604, 'recall_grapheme': 0.976226, 'recall_vowel': 0.992656, 'recall_consonant': 0.989307, 'acc_grapheme': 0.9754, 'acc_vowel': 0.993377, 'acc_consonant': 0.992256, 'loss_grapheme': 0.107763, 'loss_vowel': 0.044844, 'loss_consonant': 0.039453}


/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    2 | 0.000020 | 090112/160678 | 3.5262 | 1.3944 |
val: {'recall': 0.983226, 'recall_grapheme': 0.97561, 'recall_vowel': 0.992674, 'recall_consonant': 0.989011, 'acc_grapheme': 0.974752, 'acc_vowel': 0.993302, 'acc_consonant': 0.991684, 'loss_grapheme': 0.133167, 'loss_vowel': 0.079155, 'loss_consonant': 0.059539}
    3 | 0.000019 | 135168/160678 | 3.4017 | 1.3007 |
val: {'recall': 0.983111, 'recall_grapheme': 0.975271, 'recall_vowel': 0.99284, 'recall_consonant': 0.989063, 'acc_grapheme': 0.97525, 'acc_vowel': 0.993651, 'acc_consonant': 0.992231, 'loss_grapheme': 0.104009, 'loss_vowel': 0.034853, 'loss_consonant': 0.033734}
    5 | 0.000017 | 020480/160678 | 4.5452 | 1.8092 |
val: {'recall': 0.982948, 'recall_grapheme': 0.975272, 'recall_vowel': 0.992556, 'recall_consonant': 0.988692, 'acc_grapheme': 0.974628, 'acc_vowel': 0.993252, 'acc_consonant': 0.992032, 'loss_grapheme': 0.117416, 'loss_vowel': 0.059287, 'loss_consonant': 0.048102}
    6 | 0.000015 | 065536/160678 | 0.0066 | 1.

   35 | 0.000017 | 143360/160678 | 0.0019 | 1.5450 |
val: {'recall': 0.983287, 'recall_grapheme': 0.975601, 'recall_vowel': 0.992595, 'recall_consonant': 0.989352, 'acc_grapheme': 0.974927, 'acc_vowel': 0.993427, 'acc_consonant': 0.992231, 'loss_grapheme': 0.108547, 'loss_vowel': 0.045038, 'loss_consonant': 0.039963}
   37 | 0.000015 | 028672/160678 | 0.0032 | 1.3909 |
val: {'recall': 0.983239, 'recall_grapheme': 0.975477, 'recall_vowel': 0.992492, 'recall_consonant': 0.989509, 'acc_grapheme': 0.975425, 'acc_vowel': 0.993601, 'acc_consonant': 0.992381, 'loss_grapheme': 0.104633, 'loss_vowel': 0.034217, 'loss_consonant': 0.034157}
   38 | 0.000013 | 073728/160678 | 0.0036 | 1.4285 |
val: {'recall': 0.98337, 'recall_grapheme': 0.975778, 'recall_vowel': 0.992514, 'recall_consonant': 0.98941, 'acc_grapheme': 0.975076, 'acc_vowel': 0.993551, 'acc_consonant': 0.992256, 'loss_grapheme': 0.106842, 'loss_vowel': 0.040796, 'loss_consonant': 0.037841}
   39 | 0.000011 | 118784/160678 | 0.0037 | 1

   69 | 0.000013 | 036864/160678 | 0.0035 | 2.0978 |
val: {'recall': 0.983501, 'recall_grapheme': 0.976078, 'recall_vowel': 0.992889, 'recall_consonant': 0.98896, 'acc_grapheme': 0.975051, 'acc_vowel': 0.993476, 'acc_consonant': 0.991733, 'loss_grapheme': 0.116378, 'loss_vowel': 0.054478, 'loss_consonant': 0.04523}
   70 | 0.000011 | 081920/160678 | 0.0020 | 1.4009 |
val: {'recall': 0.983453, 'recall_grapheme': 0.975924, 'recall_vowel': 0.992875, 'recall_consonant': 0.98909, 'acc_grapheme': 0.975325, 'acc_vowel': 0.993576, 'acc_consonant': 0.992007, 'loss_grapheme': 0.109994, 'loss_vowel': 0.047794, 'loss_consonant': 0.041347}
   71 | 0.000008 | 126976/160678 | 0.0027 | 1.4938 |
val: {'recall': 0.983242, 'recall_grapheme': 0.975764, 'recall_vowel': 0.992765, 'recall_consonant': 0.988674, 'acc_grapheme': 0.975126, 'acc_vowel': 0.993476, 'acc_consonant': 0.992032, 'loss_grapheme': 0.108424, 'loss_vowel': 0.044579, 'loss_consonant': 0.039714}
   73 | 0.000006 | 012288/160678 | 0.0043 | 1.

  102 | 0.000008 | 090112/160678 | 3.2193 | 1.6613 |
val: {'recall': 0.983384, 'recall_grapheme': 0.975771, 'recall_vowel': 0.992706, 'recall_consonant': 0.98929, 'acc_grapheme': 0.975101, 'acc_vowel': 0.993551, 'acc_consonant': 0.991982, 'loss_grapheme': 0.123945, 'loss_vowel': 0.066179, 'loss_consonant': 0.051828}
  103 | 0.000006 | 135168/160678 | 3.5503 | 1.6544 |
val: {'recall': 0.983264, 'recall_grapheme': 0.975946, 'recall_vowel': 0.992679, 'recall_consonant': 0.988487, 'acc_grapheme': 0.975126, 'acc_vowel': 0.993576, 'acc_consonant': 0.992007, 'loss_grapheme': 0.112679, 'loss_vowel': 0.051317, 'loss_consonant': 0.042975}
  105 | 0.000004 | 020480/160678 | 0.1884 | 1.6048 |
val: {'recall': 0.983255, 'recall_grapheme': 0.975952, 'recall_vowel': 0.992432, 'recall_consonant': 0.988685, 'acc_grapheme': 0.975151, 'acc_vowel': 0.993452, 'acc_consonant': 0.992032, 'loss_grapheme': 0.124091, 'loss_vowel': 0.065983, 'loss_consonant': 0.051293}
  106 | 0.000002 | 065536/160678 | 0.0027 | 

  135 | 0.000004 | 143360/160678 | 0.0017 | 1.4419 |
val: {'recall': 0.983316, 'recall_grapheme': 0.975753, 'recall_vowel': 0.992939, 'recall_consonant': 0.988818, 'acc_grapheme': 0.975449, 'acc_vowel': 0.993701, 'acc_consonant': 0.992406, 'loss_grapheme': 0.105309, 'loss_vowel': 0.036222, 'loss_consonant': 0.034523}
  137 | 0.000002 | 028672/160678 | 2.1682 | 1.4081 |
val: {'recall': 0.983511, 'recall_grapheme': 0.975977, 'recall_vowel': 0.992775, 'recall_consonant': 0.989315, 'acc_grapheme': 0.975549, 'acc_vowel': 0.993601, 'acc_consonant': 0.992182, 'loss_grapheme': 0.108885, 'loss_vowel': 0.044763, 'loss_consonant': 0.039639}
  138 | 0.000001 | 073728/160678 | 0.0079 | 1.4058 |
val: {'recall': 0.983533, 'recall_grapheme': 0.976217, 'recall_vowel': 0.992976, 'recall_consonant': 0.988723, 'acc_grapheme': 0.975574, 'acc_vowel': 0.993725, 'acc_consonant': 0.992207, 'loss_grapheme': 0.106639, 'loss_vowel': 0.03966, 'loss_consonant': 0.036478}
  139 | 0.000001 | 118784/160678 | 0.0021 | 

  169 | 0.000001 | 036864/160678 | 0.0110 | 1.4008 |
val: {'recall': 0.983516, 'recall_grapheme': 0.976102, 'recall_vowel': 0.993104, 'recall_consonant': 0.988754, 'acc_grapheme': 0.975748, 'acc_vowel': 0.9939, 'acc_consonant': 0.992057, 'loss_grapheme': 0.106091, 'loss_vowel': 0.040274, 'loss_consonant': 0.037128}
  170 | 0.000001 | 081920/160678 | 0.0018 | 1.4808 |
val: {'recall': 0.983635, 'recall_grapheme': 0.976406, 'recall_vowel': 0.993101, 'recall_consonant': 0.988626, 'acc_grapheme': 0.975898, 'acc_vowel': 0.993875, 'acc_consonant': 0.992207, 'loss_grapheme': 0.104631, 'loss_vowel': 0.033412, 'loss_consonant': 0.033421}
  171 | 0.000001 | 126976/160678 | 2.7245 | 1.5091 |
val: {'recall': 0.983376, 'recall_grapheme': 0.97611, 'recall_vowel': 0.992797, 'recall_consonant': 0.988488, 'acc_grapheme': 0.975549, 'acc_vowel': 0.993725, 'acc_consonant': 0.992057, 'loss_grapheme': 0.110463, 'loss_vowel': 0.048373, 'loss_consonant': 0.041056}
  173 | 0.000002 | 012288/160678 | 0.0029 | 0.

  202 | 0.000001 | 090112/160678 | 0.0037 | 1.3208 |
val: {'recall': 0.983565, 'recall_grapheme': 0.976315, 'recall_vowel': 0.993049, 'recall_consonant': 0.988583, 'acc_grapheme': 0.975698, 'acc_vowel': 0.993775, 'acc_consonant': 0.992207, 'loss_grapheme': 0.105218, 'loss_vowel': 0.032946, 'loss_consonant': 0.033024}
  203 | 0.000002 | 135168/160678 | 0.0018 | 1.4626 |
val: {'recall': 0.98321, 'recall_grapheme': 0.97565, 'recall_vowel': 0.992999, 'recall_consonant': 0.988543, 'acc_grapheme': 0.9753, 'acc_vowel': 0.9938, 'acc_consonant': 0.992032, 'loss_grapheme': 0.106336, 'loss_vowel': 0.039182, 'loss_consonant': 0.036255}
  205 | 0.000004 | 020480/160678 | 0.0045 | 1.8145 |
val: {'recall': 0.98352, 'recall_grapheme': 0.976401, 'recall_vowel': 0.992716, 'recall_consonant': 0.988564, 'acc_grapheme': 0.975425, 'acc_vowel': 0.993651, 'acc_consonant': 0.991982, 'loss_grapheme': 0.109923, 'loss_vowel': 0.047053, 'loss_consonant': 0.04117}
  206 | 0.000006 | 065536/160678 | 3.1242 | 1.5675 

KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)

In [36]:
del model